In [ ]:
https://cdn.zephyrcms.com/2348096f-8890-41c7-b890-bcab8364e607/-/inline/yes/cip-35-testing-compressive-strength-of-concrete.pdf
https://www.bigdreadymix.com/what-is-the-standard-strength-of-concrete/#:~:text=Usually%2C%20the%20compressive%20strength%20of,10%2C000%20psi%20(70%20MPa).
https://cor-tuf.com/everything-you-need-to-know-about-concrete-strength/#:~:text=Compressive%20strength%20is%20tested%20by,Testing%20%26%20Materials)%20standard%20C39.
https://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength
https://civilread.com/find-quantity-cement-sand-aggregate/
https://civiconcepts.com/blog/concrete-mix-ratio#:~:text=One%20of%20the%20best%20ratios,%2C%20steps%2C%20and%20foundation%20walls.
https://sci-hub.se/10.1016/s0008-8846(98)00165-3
https://www.kaggle.com/datasets/sinamhd9/concrete-comprehensive-strength
https://www.analyticsvidhya.com/blog/2021/04/concrete-strength-prediction-using-machine-learning-with-python-code/
https://medium.com/data-science-at-microsoft/introduction-to-gaussian-process-regression-part-2-application-to-predicting-concrete-strength-3facef04f639#0dd7

In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 2.6 MB/s 


In [8]:
from catboost import CatBoostRegressor, Pool, cv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly
from plotly import tools
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/regression/concrete_strength/Concrete_Data.csv')
df

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [ ]:
pd.DataFrame({'Dtype': df.dtypes, 'Nunique': df.nunique(), 'Isnull': df.isnull().sum()}, index=df.columns)

,Dtype,Nunique,Isnull
Cement (component 1)(kg in a m^3 mixture),float64,278,0
Blast Furnace Slag (component 2)(kg in a m^3 mixture),float64,185,0
Fly Ash (component 3)(kg in a m^3 mixture),float64,156,0
Water (component 4)(kg in a m^3 mixture),float64,195,0
Superplasticizer (component 5)(kg in a m^3 mixture),float64,111,0
Coarse Aggregate (component 6)(kg in a m^3 mixture),float64,284,0
Fine Aggregate (component 7)(kg in a m^3 mixture),float64,302,0
Age (day),int64,14,0
"Concrete compressive strength(MPa, megapascals)",float64,845,0


In [ ]:
df.columns

Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)',
       'Concrete compressive strength(MPa, megapascals) '],
      dtype='object')

In [5]:
out_column = 'Concrete compressive strength(MPa, megapascals) '
to_remove = []
features = [c for c in df.columns if (c != out_column) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() > 2)]
binary = [c for c in features if df[c].nunique() == 2]
numerical = [col for col in features if col not in categorical + binary]
df[numerical] = df[numerical].apply(pd.to_numeric,1)
fig = go.Figure(data=go.Heatmap(z=df[numerical+[out_column]].corr(),x=numerical+[out_column],y=numerical+[out_column]))
fig.show()

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(df[features], df[out_column], test_size=0.2, random_state=42)
Y_train, Y_test = Y_train.values.ravel() + 1e-15, Y_test.values.ravel() + 1e-15
train_dataset = Pool(X_train, Y_train, feature_names=features, cat_features=categorical+binary)
test_dataset = Pool(X_test, feature_names=features, cat_features=categorical+binary)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((824, 8), (206, 8), (824,), (206,))

In [7]:
hp = [ 51.        ,   0.46626054, 412.        , 100.        ]
hp = [np.round(hp[0]).astype(int), hp[1], np.round(hp[2]).astype(int), hp[3]]
params = {"iterations": hp[0], 'learning_rate': hp[1], "min_data_in_leaf": hp[2], 'reg_lambda': hp[3], "loss_function": "MAE", "verbose": False}
cv_res = cv(train_dataset, params, fold_count=10, stratified=False, verbose=False)
cv_res

Training on fold [0/10]

bestTest = 5.089280684
bestIteration = 50

Training on fold [1/10]

bestTest = 3.414203004
bestIteration = 50

Training on fold [2/10]

bestTest = 3.647647994
bestIteration = 42

Training on fold [3/10]

bestTest = 4.241515572
bestIteration = 47

Training on fold [4/10]

bestTest = 3.183253649
bestIteration = 50

Training on fold [5/10]

bestTest = 3.813302215
bestIteration = 50

Training on fold [6/10]

bestTest = 3.570938752
bestIteration = 50

Training on fold [7/10]

bestTest = 4.525323218
bestIteration = 46

Training on fold [8/10]

bestTest = 2.922720718
bestIteration = 44

Training on fold [9/10]

bestTest = 3.123309208
bestIteration = 50



,iterations,test-MAE-mean,test-MAE-std,train-MAE-mean,train-MAE-std
0,0,20.305843,1.639087,20.302804,0.264199
1,1,12.182794,1.529738,12.130918,0.245644
2,2,8.876911,1.283518,8.511171,0.257703
3,3,7.379128,1.213245,6.892105,0.256039
4,4,6.554504,1.143971,5.921607,0.186646
5,5,6.021875,1.126173,5.297488,0.173442
6,6,5.638645,0.997879,4.842177,0.174374
7,7,5.421335,0.981157,4.554866,0.147196
8,8,5.233896,0.945751,4.333841,0.134265
9,9,5.114293,0.960950,4.155421,0.127573


In [10]:
regr = CatBoostRegressor(iterations=hp[0],  learning_rate=hp[1], min_data_in_leaf=hp[2], reg_lambda=hp[3], loss_function='MAE')
# train the model
regr.fit(train_dataset, verbose=True)
train_pred = regr.predict(X_train)
test_pred = regr.predict(X_test)

eval_df = pd.DataFrame([{'R2': r2_score(Y_train, train_pred), 'MSE': mean_squared_error(Y_train, train_pred, squared=False), 'MAE': mean_absolute_error(Y_train, train_pred)},
                        {'R2': r2_score(Y_test, test_pred), 'MSE': mean_squared_error(Y_test, test_pred, squared=False), 'MAE': mean_absolute_error(Y_test, test_pred)}], index=['TRAIN', 'TEST'])
eval_df

0:	learn: 9.4927349	total: 2.23ms	remaining: 111ms
1:	learn: 7.6388064	total: 4.79ms	remaining: 117ms
2:	learn: 6.1737228	total: 7.11ms	remaining: 114ms
3:	learn: 5.1786350	total: 9.21ms	remaining: 108ms
4:	learn: 4.7102684	total: 11.3ms	remaining: 104ms
5:	learn: 4.4364904	total: 13.3ms	remaining: 100ms
6:	learn: 4.1356510	total: 15.4ms	remaining: 97ms
7:	learn: 3.8961741	total: 17.5ms	remaining: 94ms
8:	learn: 3.7381965	total: 19.5ms	remaining: 91ms
9:	learn: 3.6000124	total: 21.6ms	remaining: 88.7ms
10:	learn: 3.4569519	total: 23.8ms	remaining: 86.5ms
11:	learn: 3.3658602	total: 25.9ms	remaining: 84.2ms
12:	learn: 3.3272504	total: 28.1ms	remaining: 82ms
13:	learn: 3.1914217	total: 30.1ms	remaining: 79.5ms
14:	learn: 3.1233374	total: 32.1ms	remaining: 77ms
15:	learn: 3.0512476	total: 34.2ms	remaining: 74.9ms
16:	learn: 2.9805135	total: 36.3ms	remaining: 72.7ms
17:	learn: 2.9329697	total: 38.4ms	remaining: 70.4ms
18:	learn: 2.7854370	total: 40.4ms	remaining: 68ms
19:	learn: 2.7155119	

,R2,MSE,MAE
TRAIN,0.965461,3.132413,1.857169
TEST,0.911327,4.780066,3.316017


In [11]:
pd.DataFrame({'True': Y_test, 'Pred': test_pred})

,True,Pred
0,52.91,51.647505
1,55.90,50.992921
2,74.50,74.280762
3,35.30,35.614882
4,10.54,10.600995
...,...,...
201,56.14,55.302087
202,18.75,25.848185
203,38.00,37.739386
204,74.36,64.587971


In [12]:
from sklearn.inspection import permutation_importance

result = permutation_importance(regr, X_test, Y_test, n_repeats=10, random_state=42, n_jobs=2, scoring='r2')
fig = go.Figure()
for i in range(X_test.shape[1]):
    fig.add_trace(go.Box(x=result['importances'][i], name=X_test.columns[i]))

fig.show()

In [ ]:
import warnings
import traceback
import dask
import numpy as np
import pandas as pd
import umap
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor as RFR, GradientBoostingRegressor as GBR
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import category_encoders as ce
import optimization.numpy_version.single_objective.continuous as co

def _cv_score(hp):
    hp = [np.round(hp[0]).astype(int), hp[1], hp[2], np.round(hp[3]).astype(int)]
    regr = RFR(n_estimators=hp[0], n_jobs=-1, min_samples_split=hp[1], min_samples_leaf=hp[2], max_features=hp[3], bootstrap=True, random_state=1)
    score = cross_val_score(regr, X_train, Y_train, cv=kf,scoring='neg_mean_absolute_error').mean()
    return hp, -score

def cv_score(hp):
    hp = [np.round(hp[0]).astype(int), hp[1], hp[2], hp[3], np.round(hp[4]).astype(int)]
    regr = GBR(n_estimators=hp[0], min_samples_split=hp[1], min_samples_leaf=hp[2], learning_rate=hp[3], max_features=hp[4], random_state=1)
    score = cross_val_score(regr, X_train, Y_train, cv=kf, scoring='neg_mean_absolute_error',).mean()
    return hp, -score

def objf(pop, dim=None):
    population, fitness = [], []
    for hp in pop:
        _hp, mse = dask.delayed(cv_score, nout=2)(hp)
        population.append(_hp), fitness.append(mse)
    population, fitness = dask.compute(population, fitness)
    population, fitness = np.asarray(population), np.asarray(fitness)
    return population, fitness  # duration =  27.99079418182373

df = pd.read_csv('/home/skyolia/Téléchargements/Concrete_Data.csv')
########################################################################################################################
out_column = 'Concrete compressive strength(MPa, megapascals) '
to_remove = []
features = [c for c in df.columns if (c != out_column) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() > 2)]
binary = [c for c in features if df[c].nunique() == 2]
numerical = [col for col in features if col not in categorical + binary]
df[numerical] = df[numerical].apply(pd.to_numeric,1)
########################################################################################################################
X_train, X_test, Y_train, Y_test = train_test_split(df[features], df[out_column], test_size=0.2, random_state=42)
Y_train, Y_test = Y_train.values.ravel() + 1e-15, Y_test.values.ravel() + 1e-15

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
########################################################################################################################
def ccf(population):
    return population

kf = KFold(n_splits=10, shuffle=True, random_state=1)

#lb, ub = [2, 1e-5, 1e-5, 1], [100, 1, 0.5, X_train.shape[1]]
lb, ub = [2, 1e-5, 1e-5, 1e-5, 1], [100, 1, 0.5, 2, X_train.shape[1]]
dim = len(lb)
optimizer = co.CPSO(objf, ccf, dim, lb, ub, 50, 5000)